In [5]:
import pandas as pd

import compute_stats as cp

df = pd.read_excel("private_data/entries_stats_01.xlsx")
df.head()

,Unnamed: 0.1,Unnamed: 0,username,entry,position,score,top_size,album,family,genre,origin,total_score,nb_votes,rarity,pop_score,edgyness
0,0,0,Moland Fengkov,mastodon – hushed and grim,1,30.000000,30,mastodon - hushed and grim,Prog & Psychedelia,Prog Metal,US,438.976293,18,0.005880,10.904634,2.751124
1,1,1,Moland Fengkov,grey aura – zwart vierkant,2,29.137931,30,grey aura - zwart vierkant,Black Metal,Post-Black Metal,NL,152.427909,7,0.005968,2.241855,13.381775
2,2,2,Moland Fengkov,papangu – holoceno,3,28.275862,30,papangu - holoceno,Prog & Psychedelia,Prog Metal,BR,77.758621,4,0.026462,0.064189,467.366471
3,3,3,Moland Fengkov,godspeed you black emperor – g_d’s pee at stat...,4,27.413793,30,godspeed you black emperor - g_d's pee at stat...,Tutoiement de l'Infini,Post-Rock,CA,93.663793,5,0.031225,0.498254,60.210307
4,4,4,Moland Fengkov,big|brave – vital,5,26.551724,30,big|brave - vital,Doom,Drone Doom - Ritual - Ambient Metal,CA,35.000000,2,0.087958,-1.075040,-27.905948


In [17]:
dfg = df.groupby(by=['username','family']).sum()

genres_list = pd.read_excel('private_data/genres_list.xlsx')

dfg.head()

Unnamed: 0.1  Unnamed: 0  position  \
username family                                                         
Alex KM  Avant-Garde & Nawakeries           639          12        13   
         Black Metal                      13604         222       238   
         Death Metal                       1587          46        48   
         Doom                              1362          35        37   
         Folksies                          1718          15        17   

                                        score  top_size  total_score  \
username family                                                        
Alex KM  Avant-Garde & Nawakeries   19.655172        30    90.283251   
         Black Metal               288.620690       480   807.752427   
         Death Metal                20.344828        60   221.099138   
         Doom                       29.827586        60   123.869048   
         Folksies                   47.068966        60   135.215517   

                                   nb_votes    rarity  pop_score     edgyness  
username family                                                                
Alex KM  Avant-Garde & Nawakeries         4  0.051189   0.290793   103.166073  
         Black Metal                     44  0.394917  -6.897939          NaN  
         Death Metal                     12  0.037470   0.455768   -31.281834  
         Doom                             7  0.123141   0.261948   -66.965095  
         Folksies                         7  0.269846  -0.293891 -1137.087217

In [24]:
dfk = pd.merge(left=dfg['total_score'], right=genres_list, on='family', how='left')

dfk.head()

,family,total_score,genre
0,Avant-Garde & Nawakeries,90.283251,Avant-Garde Metal
1,Avant-Garde & Nawakeries,90.283251,Experimental
2,Avant-Garde & Nawakeries,90.283251,Funk Rock
3,Avant-Garde & Nawakeries,90.283251,Funk Metal
4,Avant-Garde & Nawakeries,90.283251,Fusion


0        diabolizer - khalkedonian death
1                    loharano - loharano
2    borstal - at her majesty’s pleasure
3                          llnn -unmaker
4      proscriptor mcgovern's apsû - s/t
dtype: object

In [19]:
entries_df.shape

(67,)